In [182]:
#getting fantasy site lines
#removing weird lines like series, half of quarter
import pandas as pd
import datetime as dt
today = dt.datetime.today()
pp_lines = pd.read_csv(f'Lines/pp/pp_{today.year}_{today.month}_{today.day}.csv',index_col=0)
dog_lines = pd.read_csv(f'Lines/dog/doglines_{today.month}_{today.day}.csv',index_col=0)
pp_lines = pp_lines.loc[~pp_lines['league'].isin([250,84,251,231,192,193,227])]
unabated = pd.read_csv(f'Lines/unabated/unabated_{today.year}_{today.month}_{today.day}.csv',index_col=0)


In [148]:
unabated['date'] = pd.to_datetime(unabated['event_time'])
unabated

,prop_id,player,player_id,line,stat,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time,date
0,001b5ff5-4,Taurean Prince,43133,8.5,PTS+AST,3,2023-10-24 18:30:00-05:00,DEN,LAL,4,0.511732,0.488268,8.823215,2023-10-24 16:04:00,2023-10-24 18:30:00-05:00
1,001b5ff5-4,Taurean Prince,43133,9.5,PTS+AST,3,2023-10-24 18:30:00-05:00,DEN,LAL,1,0.424837,0.575163,8.823215,2023-10-24 16:04:00,2023-10-24 18:30:00-05:00
2,003085d6-5,Joel Embiid,43292,14.5,TRB + AST,3,2023-10-26 18:30:00-05:00,MIL,PHI,4,0.485273,0.514727,14.525772,2023-10-24 16:04:00,2023-10-26 18:30:00-05:00
3,007003a6-2,RJ Barrett,43290,1.5,3P,3,2023-10-25 18:00:00-05:00,NYK,BOS,11,0.532057,0.467943,1.780639,2023-10-24 16:04:00,2023-10-25 18:00:00-05:00
4,008edd5a-e,Bennedict Mathurin,43529,0.5,3P,3,2023-10-25 18:00:00-05:00,IND,WAS,4,0.655831,0.344169,1.064563,2023-10-24 16:04:00,2023-10-25 18:00:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,feffdfca-9,Fred VanVleet,43006,27.5,PTS+AST,3,2023-10-25 18:00:00-05:00,ORL,HOU,6,0.491461,0.508539,27.466344,2023-10-24 16:04:00,2023-10-25 18:00:00-05:00
2735,ff04b8f6-d,Kevon Looney,238647,17.5,PTS+TRB,3,2023-10-24 21:00:00-05:00,GSW,PHX,4,0.491841,0.508159,17.581737,2023-10-24 16:04:00,2023-10-24 21:00:00-05:00
2736,ffd4bd86-3,Trae Young,5334,35.5,PTS+AST,3,2023-10-25 18:00:00-05:00,CHA,ATL,1,0.518759,0.481241,36.512836,2023-10-24 16:04:00,2023-10-25 18:00:00-05:00
2737,ffd4bd86-3,Trae Young,5334,36.5,PTS+AST,3,2023-10-25 18:00:00-05:00,CHA,ATL,5,0.496160,0.503840,36.512836,2023-10-24 16:04:00,2023-10-25 18:00:00-05:00


In [5]:

#finding where prizepicks and underdog are different
fantasy = pp_lines.merge(dog_lines, on=['player','stat','date'],how='inner',suffixes=('_pp','_dog'))
fantasy['difference'] = ((fantasy['line_pp'] - fantasy['line_dog']))/fantasy['line_pp']
fantasy = fantasy.sort_values(by='difference')
fantasy.sample(3)

temp = fantasy.loc[(fantasy['difference'] != 0.0)]
temp = temp.loc[temp.stat != 'Hitter Fantasy Score'].sort_values(by='difference')
temp = temp[['player','line_pp','line_dog','stat', 'difference',  'date',
       'league_dog', 'team_dog' ]]
temp.to_csv(f'bets/fantasy_bets_test.csv')
temp

,player,line_pp,line_dog,stat,difference,date,league_dog,team_dog
40,Jean Butez,1.5,3.0,Goalie Saves,-1.000000,2023-10-25,FIFA,Royal Antwerp
21,Walter Benitez,1.5,3.0,Goalie Saves,-1.000000,2023-10-24,FIFA,PSV
39,Joe Hart,1.5,3.0,Goalie Saves,-1.000000,2023-10-25,FIFA,Celtic Glasgow
11,Nightfall,14.0,16.0,MAPS 1-2 Kills,-0.142857,2023-10-27,ESPORTS,BetBoom
132,Christian Wood,7.5,8.5,PTS,-0.133333,2023-10-24,NBA,LAL
...,...,...,...,...,...,...,...,...
130,Taurean Prince,10.5,9.5,PTS+TRB,0.095238,2023-10-24,NBA,LAL
621,Zyphon,31.5,28.5,MAPS 1-2 Kills,0.095238,2023-10-24,ESPORTS,Sprout
622,Zyphon,31.5,28.5,MAPS 1-2 Kills,0.095238,2023-10-24,ESPORTS,Sprout
36,Larl,14.0,12.5,MAPS 1-2 Kills,0.107143,2023-10-27,ESPORTS,Spirit


In [6]:
compare = temp.merge(unabated, on=['player','stat'],how='inner',suffixes=('','_un'))
compare

,player,line_pp,line_dog,stat,difference,date,league_dog,team_dog,prop_id,player_id,...,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time,date_un
0,Christian Wood,7.5,8.5,PTS,-0.133333,2023-10-24,NBA,LAL,96c618d2-7,43324,...,3,2023-10-24T23:30:00,DEN,LAL,2,0.497492,0.502508,7.649152,2023-10-24 10:54:00,2023-10-24 23:30:00
1,Daniel Gafford,6.0,6.5,TRB,-0.083333,2023-10-25,NBA,WAS,44459e3c-d,241452,...,3,2023-10-25T23:00:00,IND,WAS,4,0.485273,0.514727,6.573072,2023-10-24 10:54:00,2023-10-25 23:00:00
2,Austin Ekeler,52.5,56.5,Rush Yards,-0.076190,2023-10-29,NFL,LAC,fa72a9f2-d,35465,...,1,2023-10-30T00:20:00,LAC,CHI,1,0.500000,0.500000,54.684165,2023-10-24 10:54:00,2023-10-30 00:20:00
3,Austin Ekeler,52.5,56.5,Rush Yards,-0.076190,2023-10-29,NFL,LAC,fa72a9f2-d,35465,...,1,2023-10-30T00:20:00,LAC,CHI,1,0.500000,0.500000,54.684165,2023-10-24 10:54:00,2023-10-30 00:20:00
4,Isaiah Stewart,10.0,10.5,PTS+AST,-0.050000,2023-10-25,NBA,DET,504b709b-7,242076,...,3,2023-10-25T23:30:00,MIA,DET,1,0.469388,0.530612,10.426711,2023-10-24 10:54:00,2023-10-25 23:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Brandon Ingram,6.0,5.5,TRB,0.083333,2023-10-25,NBA,NOP,bac0c593-e,42496,...,3,2023-10-26T00:00:00,MEM,NOP,6,0.532138,0.467862,5.618780,2023-10-24 10:54:00,2023-10-26 00:00:00
137,Brandon Ingram,6.0,5.5,TRB,0.083333,2023-10-25,NBA,NOP,bac0c593-e,42496,...,3,2023-10-26T00:00:00,MEM,NOP,1,0.500000,0.500000,5.618780,2023-10-24 10:54:00,2023-10-26 00:00:00
138,Daniel Gafford,10.5,9.5,PTS,0.095238,2023-10-25,NBA,WAS,949bae82-a,241452,...,3,2023-10-25T23:00:00,IND,WAS,2,0.514727,0.485273,10.167249,2023-10-24 10:54:00,2023-10-25 23:00:00
139,Daniel Gafford,10.5,9.5,PTS,0.095238,2023-10-25,NBA,WAS,949bae82-a,241452,...,3,2023-10-25T23:00:00,IND,WAS,2,0.486110,0.513890,10.167249,2023-10-24 10:54:00,2023-10-25 23:00:00


In [10]:
compare = compare[['player','line_pp','line_dog','line','stat','over_prob','under_prob','date','league_dog','pred']]
temp = compare.loc[(compare.line < compare.line_pp)  & (compare.date == '2023-10-24')]
temp[['player','line_pp','line_dog','line','pred','date','stat','over_prob','under_prob']].sort_values(by='over_prob')

,player,line_pp,line_dog,line,pred,date,stat,over_prob,under_prob
51,Jamal Murray,33.0,33.5,32.5,32.993253,2023-10-24,PTS+TRB+AST,0.482543,0.517457
37,Austin Reaves,23.0,23.5,22.5,23.119242,2023-10-24,PTS+TRB+AST,0.488897,0.511103
28,Chris Paul,20.0,20.5,19.5,20.262552,2023-10-24,PTS+AST,0.498298,0.501702
105,Kentavious Caldwell-Pope,15.0,14.5,14.5,15.248677,2023-10-24,PTS+TRB+AST,0.500000,0.500000
40,Jamal Murray,26.0,26.5,25.5,26.482446,2023-10-24,PTS+TRB,0.504155,0.495845
77,Chris Paul,24.0,23.5,23.5,24.080555,2023-10-24,PTS+TRB+AST,0.507358,0.492642
129,Austin Reaves,15.5,14.5,14.5,14.813942,2023-10-24,PTS,0.508742,0.491258
34,Aaron Gordon,23.0,23.5,22.5,23.486747,2023-10-24,PTS+TRB+AST,0.510121,0.489879
88,Aaron Gordon,17.0,16.5,16.5,16.943611,2023-10-24,PTS+AST,0.514055,0.485945
111,Aaron Gordon,14.0,13.5,13.5,14.174273,2023-10-24,PTS,0.518147,0.481853


In [9]:
compare

,player,line_pp,line_dog,line,stat,over_prob,under_prob,date,league_dog,pred
0,Christian Wood,7.5,8.5,7.5,PTS,0.497492,0.502508,2023-10-24,NBA,7.649152
1,Daniel Gafford,6.0,6.5,6.5,TRB,0.485273,0.514727,2023-10-25,NBA,6.573072
2,Austin Ekeler,52.5,56.5,52.5,Rush Yards,0.500000,0.500000,2023-10-29,NFL,54.684165
3,Austin Ekeler,52.5,56.5,56.5,Rush Yards,0.500000,0.500000,2023-10-29,NFL,54.684165
4,Isaiah Stewart,10.0,10.5,10.5,PTS+AST,0.469388,0.530612,2023-10-25,NBA,10.426711
...,...,...,...,...,...,...,...,...,...,...
136,Brandon Ingram,6.0,5.5,5.5,TRB,0.532138,0.467862,2023-10-25,NBA,5.618780
137,Brandon Ingram,6.0,5.5,6.0,TRB,0.500000,0.500000,2023-10-25,NBA,5.618780
138,Daniel Gafford,10.5,9.5,9.5,PTS,0.514727,0.485273,2023-10-25,NBA,10.167249
139,Daniel Gafford,10.5,9.5,10.5,PTS,0.486110,0.513890,2023-10-25,NBA,10.167249


In [11]:

temp.loc[temp.Stat != 'Hitter Fantasy Score']

temp.loc[temp.League_dog == 'ESPORTS']

#underdog for now
fullpdf = pd.read_csv('Lines/unabated_2023_10_6.csv',index_col=0)
fullpdf.rename({'player':'Player','line':'Line','stat':'Stat'},inplace=True)
new = fullpdf.merge(pp_lines, on=['Player','Stat','Line'],how='inner').sort_values(by='o_Prob')
new.to_csv(f'bets/bets_{str(today)}.csv')
new= new.round(3)

new = new.merge(dog_lines, on=['Player','Stat'],how='inner',suffixes=('_pp','_dog'))
new['difference'] = ((new['Line_pp'] - new['Line_dog']))/new['Line_pp']
new

new.merge(temp, on=['Player','Stat','Line_pp'],how='inner')

new.columns

new[['Player', 'Stat', 'Line_pp', 'Line_dog', 'o_Prob', 'u_Prob','Date_pp','difference']].tail(20)

import yagmail
import os

attachment = os.path.abspath(f"bets/bets_{today}.csv")

send = True


if send == True:
    with open(attachment, "rb") as f:
        # send the email with the file object as an attachment

        yag = yagmail.SMTP("sharedwestover@gmail.com", oauth2_file="secrets.json")
        yag.send(subject=f"BETS! {str(today)}",attachments=f)
        
    attachment = os.path.abspath(f"bets/fantasy_bets_test.csv")

AttributeError: 'DataFrame' object has no attribute 'Stat'

In [16]:
dog_lines.loc[dog_lines.league == 'NBA']

,line,stat,player,league,team,date
339,12.5,PTS,Dejounte Murray,NBA,ATL,2023-10-16
340,19.5,PTS+TRB+AST,Dejounte Murray,NBA,ATL,2023-10-16
341,4.0,TRB,Dejounte Murray,NBA,ATL,2023-10-16
342,9.5,PTS,Bogdan Bogdanovic,NBA,ATL,2023-10-16
343,14.5,PTS+TRB+AST,Bogdan Bogdanovic,NBA,ATL,2023-10-16
347,7.5,PTS,Bruce Brown,NBA,IND,2023-10-16
348,12.5,PTS+TRB+AST,Bruce Brown,NBA,IND,2023-10-16
349,15.5,PTS,Bennedict Mathurin,NBA,IND,2023-10-16
350,22.5,PTS+TRB+AST,Bennedict Mathurin,NBA,IND,2023-10-16
351,19.5,PTS,Tyrese Maxey,NBA,PHI,2023-10-16


In [18]:
pp_lines.loc[(pp_lines.league ==7) & (pp_lines.team=='PHX')]

,player,team,line,stat,league,date,time
0,Stephen Curry,PHX,28.5,PTS,7,2023-10-24,2023-10-16 16:55:00
834,Stephen Curry,PHX,39.5,PTS+TRB+AST,7,2023-10-24,2023-10-16 16:55:00
836,Stephen Curry,PHX,28.5,PTS,7,2023-10-24,2023-10-16 16:55:00
837,Stephen Curry,PHX,5.5,TRB,7,2023-10-24,2023-10-16 16:55:00
838,Stephen Curry,PHX,4.5,3P,7,2023-10-24,2023-10-16 16:55:00
839,Stephen Curry,PHX,5.5,AST,7,2023-10-24,2023-10-16 16:55:00
845,Klay Thompson,PHX,27.5,PTS+TRB+AST,7,2023-10-24,2023-10-16 16:55:00
846,Klay Thompson,PHX,20.5,PTS,7,2023-10-24,2023-10-16 16:55:00
847,Klay Thompson,PHX,4.5,3P,7,2023-10-24,2023-10-16 16:55:00
854,Andrew Wiggins,PHX,24.5,PTS+TRB+AST,7,2023-10-24,2023-10-16 16:55:00


In [24]:
fantasy.loc[fantasy.league_dog=='NBA']

,player,team_pp,line_pp,stat,league_pp,date,time,line_dog,league_dog,team_dog,difference
46,Tobias Harris,BKN,12.5,PTS,237,2023-10-16,2023-10-16 16:55:00,13.5,NBA,PHI,-0.080000
74,Jerami Grant,PHX,13.5,PTS,237,2023-10-16,2023-10-16 16:55:00,14.5,NBA,POR,-0.074074
43,Tyrese Maxey,BKN,18.5,PTS,237,2023-10-16,2023-10-16 16:55:00,19.5,NBA,PHI,-0.054054
61,Jeremy Sochan,HOU,9.5,PTS,237,2023-10-16,2023-10-16 16:55:00,10.0,NBA,SAS,-0.052632
4,Dejounte Murray,IND,12.0,PTS,237,2023-10-16,2023-10-16 16:55:00,12.5,NBA,ATL,-0.041667
8,Bruce Brown,ATL,7.5,PTS,237,2023-10-16,2023-10-16 16:55:00,7.5,NBA,IND,0.000000
13,Bogdan Bogdanovic,IND,9.5,PTS,237,2023-10-16,2023-10-16 16:55:00,9.5,NBA,ATL,0.000000
2,Bennedict Mathurin,ATL,15.5,PTS,237,2023-10-16,2023-10-16 16:55:00,15.5,NBA,IND,0.000000
75,Jusuf Nurkic,POR,10.5,PTS,237,2023-10-16,2023-10-16 16:55:00,10.5,NBA,PHX,0.000000
73,Scoot Henderson,PHX,18.5,PTS,237,2023-10-16,2023-10-16 16:55:00,18.5,NBA,POR,0.000000


In [149]:
pp1 = pd.read_csv('Lines/pp/pp_2023_10_24.csv',index_col=0)
pp2 = pd.read_csv('Lines/pp/pp_2023_10_24_test.csv',index_col=0)

In [150]:
unabated.loc[unabated.player=='Stephen Curry']

,prop_id,player,player_id,line,stat,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time,date
17,024691b9-5,Stephen Curry,42631,35.5,PTS+TRB,3,2023-10-24 21:00:00-05:00,GSW,PHX,9,0.502312,0.497688,35.715264,2023-10-24 16:04:00,2023-10-24 21:00:00-05:00
210,11735744-b,Stephen Curry,42631,4.5,3P,3,2023-10-24 21:00:00-05:00,GSW,PHX,13,0.509538,0.490462,4.720025,2023-10-24 16:04:00,2023-10-24 21:00:00-05:00
442,2775b87d-b,Stephen Curry,42631,36.5,PTS+AST,3,2023-10-24 21:00:00-05:00,GSW,PHX,3,0.482644,0.517356,37.048709,2023-10-24 16:04:00,2023-10-24 21:00:00-05:00
443,2775b87d-b,Stephen Curry,42631,37.0,PTS+AST,3,2023-10-24 21:00:00-05:00,GSW,PHX,1,0.500000,0.500000,37.048709,2023-10-24 16:04:00,2023-10-24 21:00:00-05:00
444,2775b87d-b,Stephen Curry,42631,37.5,PTS+AST,3,2023-10-24 21:00:00-05:00,GSW,PHX,5,0.481715,0.518285,37.048709,2023-10-24 16:04:00,2023-10-24 21:00:00-05:00
551,325468f7-c,Stephen Curry,42631,29.5,PTS,3,2023-10-24 21:00:00-05:00,GSW,PHX,7,0.514173,0.485827,30.159524,2023-10-24 16:04:00,2023-10-24 21:00:00-05:00
552,325468f7-c,Stephen Curry,42631,30.5,PTS,3,2023-10-24 21:00:00-05:00,GSW,PHX,8,0.480457,0.519543,30.159524,2023-10-24 16:04:00,2023-10-24 21:00:00-05:00
719,455f156d-6,Stephen Curry,42631,4.5,TRB,3,2023-10-24 21:00:00-05:00,GSW,PHX,4,0.568343,0.431657,5.151193,2023-10-24 16:04:00,2023-10-24 21:00:00-05:00
720,455f156d-6,Stephen Curry,42631,5.0,TRB,3,2023-10-24 21:00:00-05:00,GSW,PHX,2,0.500000,0.500000,5.151193,2023-10-24 16:04:00,2023-10-24 21:00:00-05:00
721,455f156d-6,Stephen Curry,42631,5.5,TRB,3,2023-10-24 21:00:00-05:00,GSW,PHX,8,0.418316,0.581684,5.151193,2023-10-24 16:04:00,2023-10-24 21:00:00-05:00


In [151]:
pp1.dtypes

player     object
team       object
line      float64
stat       object
league      int64
date       object
time       object
dtype: object

In [152]:
unabated["event_time"].apply(lambda x: pd.to_datetime(x,utc=True))
pp1['date'] = pp1["date"].apply(lambda x: pd.to_datetime(x,utc=True))
combined = pp1.merge(unabated,how='inner', on=['player','date'])
combined

,player,team,line_x,stat_x,league,date,time_x,prop_id,player_id,line_y,stat_y,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time_y


In [42]:
combined["date_y"].apply(lambda x: pd.to_datetime(x))

0      2023-10-24 22:00:00-04:00
1      2023-10-24 22:00:00-04:00
2      2023-10-24 19:00:00-04:00
3      2023-10-24 19:00:00-04:00
4      2023-10-24 20:07:00-04:00
                  ...           
2592   2023-10-25 13:00:00-04:00
2593   2023-10-25 13:00:00-04:00
2594   2023-10-24 18:00:00-04:00
2595   2023-10-24 19:15:00-04:00
2596   2023-10-24 20:30:00-04:00
Name: date_y, Length: 2597, dtype: datetime64[ns, UTC-04:00]

In [33]:
changed = combined.loc[(combined.line_x != combined.line_y) & (combined.league==7)]
changed

,player,team,line_x,stat,league,date_x,time_x,line_y,date_y,time_y
1,Stephen Curry,PHX,28.5,PTS,7,2023-10-24,2023-10-24 10:54:00,30.5,2023-10-24T22:00:00-04:00,2023-10-24 15:23:00
63,Austin Reaves + Jamal Murray,DEN/LAL,37.5,Points (Combo),7,2023-10-24,2023-10-24 10:54:00,36.5,2023-10-24T19:30:00-04:00,2023-10-24 15:23:00
793,Nikola Jokic,LAL,35.5,PTS+AST,7,2023-10-24,2023-10-24 10:54:00,35.0,2023-10-24T19:50:00-04:00,2023-10-24 15:23:00
794,Nikola Jokic,LAL,39.0,PTS+TRB,7,2023-10-24,2023-10-24 10:54:00,38.5,2023-10-24T19:50:00-04:00,2023-10-24 15:23:00
800,Nikola Jokic,LAL,48.5,PTS+TRB+AST,7,2023-10-24,2023-10-24 10:54:00,48.0,2023-10-24T19:50:00-04:00,2023-10-24 15:23:00
801,Nikola Jokic,LAL,26.5,PTS,7,2023-10-24,2023-10-24 10:54:00,26.0,2023-10-24T19:50:00-04:00,2023-10-24 15:23:00
850,Michael Porter Jr.,LAL,21.0,PTS+TRB+AST,7,2023-10-24,2023-10-24 10:54:00,21.5,2023-10-24T19:50:00-04:00,2023-10-24 15:23:00
852,Michael Porter Jr.,LAL,13.5,PTS,7,2023-10-24,2023-10-24 10:54:00,14.5,2023-10-24T19:50:00-04:00,2023-10-24 15:23:00
856,Michael Porter Jr.,LAL,19.5,PTS+TRB,7,2023-10-24,2023-10-24 10:54:00,20.0,2023-10-24T19:50:00-04:00,2023-10-24 15:23:00
857,Michael Porter Jr.,LAL,15.5,PTS+AST,7,2023-10-24,2023-10-24 10:54:00,16.0,2023-10-24T19:50:00-04:00,2023-10-24 15:23:00


In [103]:
curry_pp = pp2.iloc[0:2].copy(deep=True)

In [104]:
curry_un = unabated.iloc[425:427].copy(deep=True)

In [153]:
curry_pp.merge(curry_un,left_on=['player','date'],right_on=['player','event_time'])

,player,team,line_x,stat_x,league,date,time_x,prop_id,player_id,line_y,stat_y,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time_y


In [154]:
curry_un

,prop_id,player,player_id,line,stat,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time
425,26ab062d-0,Zach Collins,43064,12.5,PTS,3,2023-10-26 01:30:00,SAS,DAL,5,0.517194,0.482806,13.351854,2023-10-24 15:43:00
426,26ab062d-0,Zach Collins,43064,13.5,PTS,3,2023-10-26 01:30:00,SAS,DAL,8,0.501864,0.498136,13.351854,2023-10-24 15:43:00


In [93]:
curry_pp['date'] = curry_pp['date'].apply(lambda x: pd.to_datetime(x))
curry_un['event_time'] = curry_un['event_time'].apply(lambda x: pd.to_datetime(x))

In [108]:
curry_pp

,player,team,line,stat,league,date,time
0,Austin Reaves,DEN,15.5,PTS,7,2023-10-24T16:30:00-04:00,2023-10-24 15:23:00
1,Stephen Curry,PHX,28.5,PTS,7,2023-10-24T22:00:00-04:00,2023-10-24 15:23:00


In [155]:
curry_pp.dtypes

player     object
team       object
line      float64
stat       object
league      int64
date       object
time       object
dtype: object

In [156]:
curry_un.dtypes

prop_id        object
player         object
player_id       int64
line          float64
stat           object
league_id       int64
event_time     object
opp            object
Team           object
count           int64
over_prob     float64
under_prob    float64
pred          float64
time           object
dtype: object

In [116]:
dfa = pd.DataFrame([[1,'2023-10-24T16:30:00-04:00'],[2,'2023-10-24T16:35:00-04:00']],columns=['a','date'])
dfa

,a,date
0,1,2023-10-24T16:30:00-04:00
1,2,2023-10-24T16:35:00-04:00


In [123]:
dfb = pd.DataFrame([[4,'2023-10-24T15:30:00-05:00'],[2,'2023-10-24T17:45:00-05:00']],columns=['b','date'])
dfa

,a,date
0,1,2023-10-24T16:30:00-04:00
1,2,2023-10-24T16:35:00-04:00


In [124]:
dfa.merge(dfb,on='date')

,a,date,b


In [137]:
dfa['date'] = dfa['date'].apply(lambda x: pd.to_datetime(x))
dfa['date'] = dfa['date'].dt.tz_convert('America/Chicago')

In [128]:
import pytz
set(pytz.all_timezones_set) 

set()

In [130]:
print(pytz.common_timezones)

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'America/Adak', 'America/Anchorage', 'Amer

In [138]:
dfa

,a,date
0,1,2023-10-24 15:30:00-05:00
1,2,2023-10-24 15:35:00-05:00


In [246]:
unabated = pd.read_csv(f'Lines/unabated/unabated_{today.year}_{today.month}_{today.day}.csv',index_col=0,parse_dates=['event_time'])
pp_lines = pd.read_csv(f'Lines/pp/pp_{today.year}_{today.month}_{today.day}_test.csv',index_col=0,parse_dates=['date'])
pp_lines = pp_lines.loc[~pp_lines['league'].isin([250,84,251,231,192,193,227])]

In [245]:
unabated

,prop_id,player,player_id,line,stat,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time
0,00130723-4,Ausar Thompson,417201,15.5,PTS+TRB,3,2023-10-25 18:30:00-05:00,MIA,DET,5,0.489637,0.510363,15.790253,2023-10-24 16:56:00
1,00130723-4,Ausar Thompson,417201,16.5,PTS+TRB,3,2023-10-25 18:30:00-05:00,MIA,DET,2,0.468253,0.531747,15.790253,2023-10-24 16:56:00
2,0015915b-4,Jusuf Nurkic,43154,0.5,3P,3,2023-10-24 21:00:00-05:00,GSW,PHX,5,0.532583,0.467417,0.759035,2023-10-24 16:56:00
3,003b2865-f,LaMelo Ball,43460,23.5,PTS,3,2023-10-25 18:00:00-05:00,CHA,ATL,14,0.498111,0.501889,23.644365,2023-10-24 16:56:00
4,003e0b52-2,Saddiq Bey,241990,0.5,STL,3,2023-10-25 18:00:00-05:00,CHA,ATL,1,0.572148,0.427852,0.847108,2023-10-24 16:56:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,ffed8d59-5,Jalen Williams,43129,3.5,AST,3,2023-10-25 19:00:00-05:00,CHI,OKC,1,0.440513,0.559487,3.468344,2023-10-24 16:48:00
2735,ffed8d59-5,Jalen Williams,241037,3.5,AST,3,2023-10-25 19:00:00-05:00,CHI,OKC,5,0.458927,0.541073,3.468344,2023-10-24 16:48:00
2736,fff8de04-1,Alperen Sengun,43037,0.5,BLK,3,2023-10-25 18:00:00-05:00,ORL,HOU,4,0.594480,0.405520,0.900856,2023-10-24 16:48:00
2737,fffda7fa-6,Keldon Johnson,242025,28.5,PTS+TRB+AST,3,2023-10-25 20:30:00-05:00,SAS,DAL,10,0.512959,0.487041,28.982305,2023-10-24 16:48:00


In [265]:
compare = unabated.merge(pp_lines[['prop_id','line']], on='prop_id',suffixes=['_un','_pp'])

In [266]:
compare

,prop_id,player,player_id,line_un,stat,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time,line_pp
0,0136fdcc-b,Mikal Bridges,43092,30.5,PTS+TRB,3,2023-10-25 18:30:00-05:00,BKN,CLE,8,0.508728,0.491272,30.801228,2023-10-24 16:56:00,30.5
1,0136fdcc-b,Mikal Bridges,43092,30.5,PTS+TRB,3,2023-10-25 18:30:00-05:00,BKN,CLE,8,0.508728,0.491272,30.801228,2023-10-24 16:48:00,30.5
2,064f654a-e,Zay Flowers,410752,60.5,Receiving Yards,1,2023-10-29 15:25:00-05:00,ARI,BAL,1,0.500000,0.500000,60.685029,2023-10-24 16:56:00,60.5
3,064f654a-e,Zay Flowers,410752,60.5,Receiving Yards,1,2023-10-29 15:25:00-05:00,ARI,BAL,1,0.500000,0.500000,60.685029,2023-10-24 16:48:00,60.5
4,07a50bef-3,Harrison Barnes,42605,12.5,PTS,3,2023-10-25 20:00:00-05:00,UTA,SAC,6,0.518581,0.481419,13.071424,2023-10-24 16:56:00,13.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,ff86a682-5,Ben Simmons,43313,14.5,PTS+AST,3,2023-10-25 18:30:00-05:00,BKN,CLE,2,0.494920,0.505080,14.005784,2023-10-24 16:56:00,13.5
460,ff86a682-5,Ben Simmons,43313,13.5,PTS+AST,3,2023-10-25 18:30:00-05:00,BKN,CLE,5,0.510152,0.489848,14.005784,2023-10-24 16:48:00,13.5
461,ff86a682-5,Ben Simmons,43313,14.5,PTS+AST,3,2023-10-25 18:30:00-05:00,BKN,CLE,2,0.494920,0.505080,14.005784,2023-10-24 16:48:00,13.5
462,ffce44fc-a,Nic Claxton,243114,9.5,TRB,3,2023-10-25 18:30:00-05:00,BKN,CLE,11,0.500834,0.499166,9.673805,2023-10-24 16:56:00,9.5


In [270]:
compare['diff'] = (compare.line_pp - compare.pred) / compare.line_pp

In [276]:
compare.loc[(compare['count'] > 1) & (compare.league_id == 3)].sort_values(by='diff')

,prop_id,player,player_id,line_un,stat,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time,line_pp,diff
439,f1017604-7,Giannis Antetokounmpo,43416,0.5,3P,3,2023-10-26 18:30:00-05:00,MIL,PHI,3,0.508380,0.491620,0.708490,2023-10-24 16:48:00,0.5,-0.416979
438,f1017604-7,Giannis Antetokounmpo,43416,0.5,3P,3,2023-10-26 18:30:00-05:00,MIL,PHI,3,0.508380,0.491620,0.708490,2023-10-24 16:56:00,0.5,-0.416979
12,095ae7cc-5,Andrew Wiggins,43492,0.5,BLK,3,2023-10-24 21:00:00-05:00,GSW,PHX,6,0.506923,0.493077,0.705546,2023-10-24 16:56:00,0.5,-0.411091
13,095ae7cc-5,Andrew Wiggins,43492,0.5,BLK,3,2023-10-24 21:00:00-05:00,GSW,PHX,6,0.506923,0.493077,0.705546,2023-10-24 16:48:00,0.5,-0.411091
177,600e25ed-8,Brook Lopez,43530,0.5,STL,3,2023-10-26 18:30:00-05:00,MIL,PHI,3,0.500000,0.500000,0.691677,2023-10-24 16:48:00,0.5,-0.383355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,ef6d4cf1-5,Mikal Bridges,43092,25.5,PTS,3,2023-10-25 18:30:00-05:00,BKN,CLE,6,0.500688,0.499312,26.147413,2023-10-24 16:48:00,26.5,0.013305
7,07a50bef-3,Harrison Barnes,42605,13.5,PTS,3,2023-10-25 20:00:00-05:00,UTA,SAC,3,0.487316,0.512684,13.071424,2023-10-24 16:48:00,13.5,0.031746
6,07a50bef-3,Harrison Barnes,42605,12.5,PTS,3,2023-10-25 20:00:00-05:00,UTA,SAC,6,0.518581,0.481419,13.071424,2023-10-24 16:48:00,13.5,0.031746
5,07a50bef-3,Harrison Barnes,42605,13.5,PTS,3,2023-10-25 20:00:00-05:00,UTA,SAC,3,0.487316,0.512684,13.071424,2023-10-24 16:56:00,13.5,0.031746


In [277]:
latest_entries = compare.sort_values('event_time', ascending=False).groupby('prop_id').first().reset_index()


In [295]:
latest_entries[(latest_entries['count'] > 1)].sort_values(by='over_prob').head(20)

,prop_id,player,player_id,line_un,stat,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time,line_pp,diff
24,3795475c-7,Stephen Curry,42631,5.5,TRB,3,2023-10-24 21:00:00-05:00,GSW,PHX,8,0.418316,0.581684,5.151193,2023-10-24 16:48:00,5.0,-0.030239
72,87bbb716-c,Stephen Curry,42631,6.5,AST,3,2023-10-24 21:00:00-05:00,GSW,PHX,6,0.431446,0.568554,6.160524,2023-10-24 16:48:00,6.0,-0.026754
18,250633fe-2,Damian Lillard,42720,7.5,AST,3,2023-10-26 18:30:00-05:00,MIL,PHI,3,0.438653,0.561347,7.218740,2023-10-24 16:48:00,7.0,-0.031249
88,b0bda13f-a,Joel Embiid,43292,4.5,AST,3,2023-10-26 18:30:00-05:00,MIL,PHI,3,0.448006,0.551994,4.339376,2023-10-24 16:56:00,4.0,-0.084844
123,f0840c02-b,Spencer Dinwiddie,51,18.5,PTS,3,2023-10-25 18:30:00-05:00,BKN,CLE,5,0.450626,0.549374,17.819357,2023-10-24 16:48:00,18.0,0.010036
57,72f7ddd2-d,Klay Thompson,42621,4.5,TRB,3,2023-10-24 21:00:00-05:00,GSW,PHX,13,0.456451,0.543549,4.423445,2023-10-24 16:48:00,4.0,-0.105861
73,8d6c4e0c-6,Chris Paul,43094,12.5,PTS,3,2023-10-24 21:00:00-05:00,GSW,PHX,5,0.463409,0.536591,12.065242,2023-10-24 16:56:00,12.0,-0.005437
22,2c79286a-a,Stephen Curry,42631,37.5,PTS+AST,3,2023-10-24 21:00:00-05:00,GSW,PHX,5,0.481715,0.518285,37.048709,2023-10-24 16:48:00,37.0,-0.001316
2,07a50bef-3,Harrison Barnes,42605,13.5,PTS,3,2023-10-25 20:00:00-05:00,UTA,SAC,3,0.487316,0.512684,13.071424,2023-10-24 16:48:00,13.5,0.031746
119,e66e3913-6,Kevon Looney,238647,21.5,PTS+TRB+AST,3,2023-10-24 21:00:00-05:00,GSW,PHX,4,0.488596,0.511404,21.351859,2023-10-24 16:48:00,20.5,-0.041554


In [303]:
latest_entries.loc[latest_entries.opp=='DEN']

,prop_id,player,player_id,line_un,stat,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time,line_pp,diff
27,3dcf0e34-1,Travis Kelce,36266,81.5,Receiving Yards,1,2023-10-29 15:25:00-05:00,DEN,KC,2,0.5,0.5,81.687895,2023-10-24 16:56:00,81.5,-0.002305
77,925524e8-4,Isiah Pacheco,38653,65.5,Rush Yards,1,2023-10-29 15:25:00-05:00,DEN,KC,1,0.5,0.5,67.185832,2023-10-24 16:56:00,68.5,0.019185


In [301]:
unabated.loc[unabated.opp=='DEN']

,prop_id,player,player_id,line,stat,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time
6,009ca00b-0,D'Angelo Russell,42709,1.5,BLK + STL,3,2023-10-24 18:30:00-05:00,DEN,LAL,4,0.362712,0.637288,1.271764,2023-10-24 16:56:00
26,02eafbb2-5,Anthony Davis,43392,3.5,BLK + STL,3,2023-10-24 18:30:00-05:00,DEN,LAL,5,0.433248,0.566752,3.365074,2023-10-24 16:56:00
44,04998a33-3,Gabe Vincent,42858,0.5,STL,3,2023-10-24 18:30:00-05:00,DEN,LAL,2,0.381625,0.618375,0.481081,2023-10-24 16:56:00
46,057b5133-9,Rui Hachimura,42896,0.5,BLK,3,2023-10-24 18:30:00-05:00,DEN,LAL,3,0.349632,0.650368,0.430619,2023-10-24 16:56:00
47,05dab62b-f,Gabe Vincent,42858,8.5,PTS+AST,3,2023-10-24 18:30:00-05:00,DEN,LAL,3,0.482189,0.517811,8.535782,2023-10-24 16:56:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,fbf333c5-9,Austin Reaves,42647,0.5,BLK + STL,3,2023-10-24 18:30:00-05:00,DEN,LAL,4,0.619325,0.380675,0.963814,2023-10-24 16:48:00
2700,fce6c6ed-d,Austin Reaves,42647,0.5,STL,3,2023-10-24 18:30:00-05:00,DEN,LAL,5,0.521145,0.478855,0.734661,2023-10-24 16:48:00
2713,fe361ec7-1,Aaron Gordon,42722,0.5,BLK,3,2023-10-24 18:30:00-05:00,DEN,LAL,5,0.463413,0.536587,0.621005,2023-10-24 16:48:00
2721,feedc51d-9,Aaron Gordon,42722,8.5,TRB + AST,3,2023-10-24 18:30:00-05:00,DEN,LAL,7,0.544061,0.455939,9.032544,2023-10-24 16:48:00


In [306]:
unabated.loc[unabated.player_id==42709].sort_values(by='over_prob')

,prop_id,player,player_id,line,stat,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,time
6,009ca00b-0,D'Angelo Russell,42709,1.5,BLK + STL,3,2023-10-24 18:30:00-05:00,DEN,LAL,4,0.362712,0.637288,1.271764,2023-10-24 16:56:00
6,009ca00b-0,D'Angelo Russell,42709,1.5,BLK + STL,3,2023-10-24 18:30:00-05:00,DEN,LAL,4,0.362712,0.637288,1.271764,2023-10-24 16:48:00
477,2f0bb1d7-e,D'Angelo Russell,42709,0.5,BLK,3,2023-10-24 18:30:00-05:00,DEN,LAL,4,0.366544,0.633456,0.456869,2023-10-24 16:48:00
477,2f0bb1d7-e,D'Angelo Russell,42709,0.5,BLK,3,2023-10-24 18:30:00-05:00,DEN,LAL,4,0.366544,0.633456,0.456869,2023-10-24 16:56:00
1482,8cb87204-9,D'Angelo Russell,42709,2.5,3P,3,2023-10-24 18:30:00-05:00,DEN,LAL,1,0.374721,0.625279,2.066197,2023-10-24 16:48:00
1482,8cb87204-9,D'Angelo Russell,42709,2.5,3P,3,2023-10-24 18:30:00-05:00,DEN,LAL,1,0.374721,0.625279,2.066197,2023-10-24 16:56:00
1028,60d223b5-c,D'Angelo Russell,42709,5.5,AST,3,2023-10-24 18:30:00-05:00,DEN,LAL,12,0.450881,0.549119,5.354999,2023-10-24 16:48:00
1028,60d223b5-c,D'Angelo Russell,42709,5.5,AST,3,2023-10-24 18:30:00-05:00,DEN,LAL,12,0.450881,0.549119,5.354999,2023-10-24 16:56:00
1535,90fd5aad-8,D'Angelo Russell,42709,22.5,PTS+TRB+AST,3,2023-10-24 18:30:00-05:00,DEN,LAL,2,0.470230,0.529770,21.824592,2023-10-24 16:48:00
1535,90fd5aad-8,D'Angelo Russell,42709,22.5,PTS+TRB+AST,3,2023-10-24 18:30:00-05:00,DEN,LAL,2,0.470230,0.529770,21.824592,2023-10-24 16:56:00
